<a href="https://colab.research.google.com/github/SiwawawaXD/LLMclassification/blob/main/pythonCallChatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import glob
import os
import numpy as np
from scipy.interpolate import interp1d

# === 1. Define where your spectra CSVs are ===
data_paths = {
    "Acrylic": "/content/drive/MyDrive/CsvStandard50Spectra/Acrylic/*.CSV",
    "Cellulose": "/content/drive/MyDrive/CsvStandard50Spectra/Cellulose/*.CSV",
    "ENR": "/content/drive/MyDrive/CsvStandard50Spectra/ENR/*.CSV",
    "EPDM": "/content/drive/MyDrive/CsvStandard50Spectra/EPDM/*.CSV",
    "HDPE": "/content/drive/MyDrive/CsvStandard50Spectra/HDPE/*.CSV",
    "LDPE": "/content/drive/MyDrive/CsvStandard50Spectra/LDPE/*.CSV",
    "Nylon": "/content/drive/MyDrive/CsvStandard50Spectra/Nylon/*.CSV",
    "PBAT": "/content/drive/MyDrive/CsvStandard50Spectra/PBAT/*.CSV",
    "PBS": "/content/drive/MyDrive/CsvStandard50Spectra/PBS/*.CSV",
    "PC": "/content/drive/MyDrive/CsvStandard50Spectra/PC/*.CSV",
    "PEEK": "/content/drive/MyDrive/CsvStandard50Spectra/PEEK/*.CSV",
    "PEI": "/content/drive/MyDrive/CsvStandard50Spectra/PEI/*.CSV",
    "PET": "/content/drive/MyDrive/CsvStandard50Spectra/PET/*.CSV",
    "PLA": "/content/drive/MyDrive/CsvStandard50Spectra/PLA/*.CSV",
    "PMMA": "/content/drive/MyDrive/CsvStandard50Spectra/PMMA/*.CSV",
    "POM": "/content/drive/MyDrive/CsvStandard50Spectra/POM/*.CSV",
    "PP": "/content/drive/MyDrive/CsvStandard50Spectra/PP/*.CSV",
    "PS": "/content/drive/MyDrive/CsvStandard50Spectra/PS/*.CSV",
    "PTFE": "/content/drive/MyDrive/CsvStandard50Spectra/PTFE/*.CSV",
    "PU": "/content/drive/MyDrive/CsvStandard50Spectra/PU/*.CSV",
    "PVA": "/content/drive/MyDrive/CsvStandard50Spectra/PVA/*.CSV",
    "PVC": "/content/drive/MyDrive/CsvStandard50Spectra/PVC/*.CSV",
}

jsonl_out = "training_data.jsonl"

with open(jsonl_out, "w") as f_out:
    for label, pattern in data_paths.items():
        num = 1
        for file in glob.glob(pattern):
            if num < 6 :
                # === 2. Read CSV (no header) ===
                df = pd.read_csv(file, header=None, names=["x", "y"])

                # === 3. Extract x,y pairs ===
                xy_pairs = df[["x", "y"]].values.tolist()

                # === 4. Downsample (optional, comment out to keep all points) ===
                step = max(1, len(xy_pairs) // 10000)  # keep ~10000 points
                xy_pairs = xy_pairs[::5]  # can change step variable to exact number ex. change to 2 to make it skip 1 point

                # === 5. Format prompt+completion ===
                item = [
                    {"messages" : [
                        {"role" : "system", "content" : "You are an expert microplastic type classifier. The first column is X-axis and the second column is Y-axis. Classify the unknown and return only the microplastic type classname. You will be given 5 samples of each microplastic type, Use them as reference for predicting what user input."},
                        {"role" : "user", "content" : f"what is the microplastic type of the graph with the following points? there are  {xy_pairs}"},
                        {"role" : "assistant", "content" : f"{label}"}
                    ]}
                ]
                f_out.write(json.dumps(item[0]) + "\n")
                num +=1

print(f"Training data saved to {jsonl_out}")

Training data saved to training_data.jsonl


In [ ]:
from openai import OpenAI
from google.colab import userdata
# Initialize the client
# The client automatically reads the OPENAI_API_KEY environment variable
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# Upload the training file
file_upload_response = client.files.create(
  file=open("training_data.jsonl", "rb"), # Your file path
  purpose="fine-tune"
)

# Get the file ID
training_file_id = file_upload_response.id
print(f"File uploaded successfully with ID: {training_file_id}")

File uploaded successfully with ID: file-Wo5bwPZWsirv1cPiustLbF


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# The training_file_id from the previous step
training_file_id = "file-Wo5bwPZWsirv1cPiustLbF" # <-- Replace with your actual file ID

# Create the fine-tuning job
fine_tuning_job = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  model="gpt-4.1-nano-2025-04-14", # The base model you want to fine-tune
  suffix="Microplastic Classifier 1 trainset", # (Optional) A custom name for your model
  hyperparameters={"n_epochs": 3} # <--- Add this line to set the number of epochs
)


# Get the job ID
job_id = fine_tuning_job.id
print(f"Fine-tuning job created successfully with ID: {job_id}")

Fine-tuning job created successfully with ID: ftjob-92EdMxoCwlwKm35h62CDNEeb


In [ ]:
client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-92EdMxoCwlwKm35h62CDNEeb', created_at=1759269505, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=3), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XCHqF2z00bEi63AZd5QnYvjt', result_files=[], seed=1383794115, status='validating_files', trained_tokens=None, training_file='file-Wo5bwPZWsirv1cPiustLbF', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=3))), user_provided_suffix='Microplastic Classifier 1 trainset', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [ ]:
fine_tuning_job

FineTuningJob(id='ftjob-92EdMxoCwlwKm35h62CDNEeb', created_at=1759269505, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=3), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XCHqF2z00bEi63AZd5QnYvjt', result_files=[], seed=1383794115, status='validating_files', trained_tokens=None, training_file='file-Wo5bwPZWsirv1cPiustLbF', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=3))), user_provided_suffix='Microplastic Classifier 1 trainset', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [3]:
import os
import pandas as pd
import csv
import time
from openai import OpenAI
from google.colab import userdata

# Define the directory path
directory = "/content/drive/MyDrive/CsvStandard10Spectra/"

# CSV file to store results
csv_file = "chatgpt_responses.csv"

# Create CSV file with header if it doesn't exist
try:
    with open(csv_file, 'x', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Response"])
except FileExistsError:
    pass  # File already exists

# Loop through files with error handling
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # Only process files (not directories) and text files
    #if os.path.isfile(file_path) and filename.endswith(('.txt', '.CSV', '.py', '.md')):
    num = 0
    try:
        #print(f"\n=== Content of {filename} ===")
        for item in os.listdir(file_path):
          if num < 10 :
            print(item,end="")
            message = ""
            with open(file_path+'/'+item, 'r', encoding='utf-8') as file:
              #content = file.read()
              #print(content[:500])  # Print first 500 characters
              #if len(content) > 500:
              #    print("... (content truncated)")

              df = pd.read_csv(file, header=None, names=["x", "y"])
              #print(file)
              # === 3. Extract x,y pairs ===
              xy_pairs = df[["x", "y"]].values.tolist()

              # === 4. Downsample (optional, comment out to keep all points) ===
              step = max(1, len(xy_pairs) // 10000)
              xy_pairs = xy_pairs[::5]

              # === 5. Format prompt+completion ===
              messages = f"This is unknown{num} . {xy_pairs}"

            item = [
              {"role": "system", "content": "You are an expert microplastic type classifier. The first column is X-axis and the second column is Y-axis. Classify the unknown and return only the microplastic type classname"},
              {"role" : "user", "content" : messages}
            ]
            num += 1
            #print(item)

            client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

            # Your custom model ID
            fine_tuned_model_id = "ft:gpt-4.1-nano-2025-04-14:personal:microplastic-classifier-1-trainset:CLeuq3GA"

            completion = client.chat.completions.create(
            model=fine_tuned_model_id,
            messages=item
            )

            # Get response and timestamp
            response = completion.choices[0].message.content
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

            # Save to CSV
            with open(csv_file, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([timestamp, response])

            print(f"[{timestamp}] Response saved")

            # Wait 20 seconds
            time.sleep(18)
          else :
            break

    except Exception as e:
        print(f"Error reading {filename}: {e}")



SD_EPDM_1.CSV[2025-10-01 02:27:20] Response saved
SD_EPDM_2.CSV[2025-10-01 02:27:41] Response saved
SD_EPDM_3.CSV[2025-10-01 02:28:02] Response saved
SD_EPDM_4.CSV[2025-10-01 02:28:23] Response saved
SD_EPDM_5.CSV[2025-10-01 02:28:44] Response saved
SD_EPDM_6.CSV[2025-10-01 02:29:05] Response saved
SD_EPDM_7.CSV[2025-10-01 02:29:26] Response saved
SD_EPDM_8.CSV[2025-10-01 02:29:47] Response saved
SD_EPDM_9.CSV[2025-10-01 02:30:07] Response saved
SD_EPDM_10.CSV[2025-10-01 02:30:28] Response saved
SD_PVC_1.CSV[2025-10-01 02:30:50] Response saved
SD_PVC_2.CSV[2025-10-01 02:31:10] Response saved
SD_PVC_3.CSV[2025-10-01 02:31:31] Response saved
SD_PVC_4.CSV[2025-10-01 02:31:52] Response saved
SD_PVC_5.CSV[2025-10-01 02:32:12] Response saved
SD_PVC_6.CSV[2025-10-01 02:32:33] Response saved
SD_PVC_7.CSV[2025-10-01 02:32:54] Response saved
SD_PVC_8.CSV[2025-10-01 02:33:15] Response saved
SD_PVC_9.CSV[2025-10-01 02:33:36] Response saved
SD_PVC_10.CSV[2025-10-01 02:33:56] Response saved
SD_HDPE_